# GPX Viewer

This app lets you to display a track from a GPX file recorded with a GPS device.

In [1]:
import sys, os, io
from ipywidgets import Button, HTML, HBox, FileUpload, Label, Output, Layout, Image

sys.path.append(os.getcwd() + '/src')

from sailing_session import SailingSession
from sailing_graphs import *


In [2]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'./img/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [4]:
def show_examples(output):
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @output.capture()
        def on_example_clicked(change):
            # output.clear_output()
            session = SailingSession(filename)
            
            with output:
                map_polar_buttons(session)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button
    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

In [13]:
def map_polar_buttons(session, filename=""):
    def map_button():
        @out.capture()
        def map_button_clicked(change):
            out.clear_output()
            with out:
                chart = ipyleaflet_chart(session)
                graph = bqplot_graph(session)
                link_chart_graph(chart, graph, session)
                display(chart)
                display(graph)

        button = Button(description="Show Map")
        button.on_click(map_button_clicked)
        return button

    def polar_button():
        @out.capture()
        def polar_button_clicked(change):
            out.clear_output()
            with out:
                render_polar(session)

        button = Button(description="Show Polar")
        button.on_click(polar_button_clicked)
        return button
    
    buttons = [map_button(), polar_button()]
    display(HBox(buttons, layout=Layout(flex_flow='row', align_items='center')))

In [14]:
def show_uploader(output):
    uploader = FileUpload(accept='.gpx, .csv', multiple=False)

    def handle_upload(change):
        with output:
            display(HBox([Label(value='Loading...')], layout=Layout(flex_flow='row', align_items='center')))
        with open("tmp/session.csv", "w+b") as f:
            f.write(uploader.data[0])
        with output:
            session = SailingSession("tmp/session.csv")
            map_polar_buttons(session)
        
    uploader.observe(handle_upload, names='value')
    display(uploader)


If you don't have a GPX file, try with one of the following examples:

In [15]:
out = Output()

show_examples(out)

In [16]:
map_polar_out = Output()
show_uploader(map_polar_out)

FileUpload(value={}, accept='.gpx, .csv', description='Upload')

In [17]:
map_polar_out

Output()

Output()